In [1]:
import pandas as pd

In [2]:
!pip install fastparquet

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [5]:
#Training data 
dft=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
print("the january data has " +str(len(dft.columns)))

the january data has 19


In [6]:
#calculate duration using guide notebook
dft['duration'] = dft.tpep_dropoff_datetime - dft.tpep_pickup_datetime
dft.duration = dft.duration.apply(lambda td: td.total_seconds() / 60)
print("the standard deviation for january is " + str(dft.duration.std()))

the standard deviation for january is 42.59435124195458


In [7]:
#we drop outliers using the same principle from the notebook
dftf = dft[(dft.duration >= 1) & (dft.duration <= 60)]
prleft=dftf.shape[0]/dft.shape[0]
print("The percentaje left is " + str(prleft*100)+"%")


The percentaje left is 98.1220282212598%


In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dftf[categorical] = dftf[categorical].astype(str)
#With the hot encoding we have 2xn matrix

/tmp/ipykernel_26293/1488072264.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftf[categorical] = dftf[categorical].astype(str)


In [9]:
train_dicts = dftf[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = dftf[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

np.float64(7.658396898909143)

In [10]:
#Preparing validation data
dfv=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
dfv['duration'] = dfv.tpep_dropoff_datetime - dfv.tpep_pickup_datetime
dfv.duration = dfv.duration.apply(lambda td: td.total_seconds() / 60)
dfv = dfv[(dfv.duration >= 1) & (dfv.duration <= 60)]
dfv[categorical] = dfv[categorical].astype(str)

In [11]:
val_dicts = dfv[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred = lr.predict(X_val)
y_val = dfv[target].values
root_mean_squared_error(y_val, y_pred)

: 